### Big df course project
<strong>T4: Initial EDA; Common routes analysis</strong>

Jovana Videnovic & Haris Kupinic

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import os
import duckdb
import geopandas as gpd
import numpy as np
import time
import dask.dataframe as dd
import duckdb
from dask_sql import Context

In [ ]:
import seaborn as sb

In [ ]:
from dask.distributed import Client, LocalCluster
from pathlib import Path

In [ ]:
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

plt.rcParams['axes.facecolor']='w'

### Loading the df

In [ ]:
service_type = "yellow"

In [ ]:
part_df_path = Path("/d/hpc/projects/FRI/bigdata/students/jv8043/partitioned_data_new")
vis_path = Path("/d/hpc/home/jv8043/BD/project/T4/T4_vis") / service_type
tables_path = Path("/d/hpc/home/jv8043/BD/project/T4/T4_tables") / service_type

In [ ]:
os.makedirs(vis_path, exist_ok=True)
os.makedirs(tables_path, exist_ok=True)

In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit='32GB')
client = Client(cluster)

In [ ]:
client

In [ ]:
g_df = dd.read_parquet(part_df_path / "green", engine="pyarrow", assume_missing=True)
fhvhv_df = dd.read_parquet(part_df_path / "fhvhv", engine="pyarrow", assume_missing=True)
g_df = dd.read_parquet(part_df_path / "yellow", engine="pyarrow", assume_missing=True)

In [ ]:
g_df = dd.read_parquet(part_df_path / "yellow", engine="pyarrow", assume_missing=True)

In [ ]:
g_tables_path = Path("/d/hpc/home/jv8043/BD/project/T4/T4_tables") / "green"
fhvhv_tables_path = Path("/d/hpc/home/jv8043/BD/project/T4/T4_tables") / "fhvhv"
y_tables_path = Path("/d/hpc/home/jv8043/BD/project/T4/T4_tables") / "yellow"

In [ ]:
display(g_df.head(3))

In [ ]:
taxi_zone_lookup = pd.read_csv("/d/hpc/home/jv8043/BD/project/add_data/taxi_zone_lookup.csv")

In [ ]:
jfk_lat = 40.63983
jfk_lon = -73.77874
ues_lat = 40.7641
ues_lon = -73.956

In [ ]:
# Step 1: Filter for rides with exact matching pickup and dropoff coordinates
route_ddf = g_df[
    (g_df['pickup_lat'] == jfk_lat) & 
    (g_df['pickup_lon'] == jfk_lon) & 
    (g_df['dropoff_lat'] == ues_lat) & 
    (g_df['dropoff_lon'] == ues_lon)
]

In [ ]:
# Step 2: Extract year from pickup_datetime
route_ddf['year'] = route_ddf['pickup_datetime'].dt.year

In [ ]:
# Step 3: Group by year and compute average fare amount
avg_fare_per_year = route_ddf.groupby('year')['fare_amount'].mean().compute()

In [ ]:
# save to csv
avg_fare_per_year.to_csv(y_tables_path / "yellow_avg_fare_per_year.csv", index=True)

In [ ]:
# Define function to assign season
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

# Extract month and map to season
route_ddf['month'] = route_ddf['pickup_datetime'].dt.month
route_ddf['season'] = route_ddf['month'].map(get_season, meta=('month', 'str'))

# Group by season and compute average fare
avg_fare_per_season = route_ddf.groupby('season')['fare_amount'].mean().compute()

# Sort for better readability (optional)
avg_fare_per_season = avg_fare_per_season.loc[['Winter', 'Spring', 'Summer', 'Fall']]

# Display result
print(avg_fare_per_season)
avg_fare_per_season.to_csv(g_tables_path / "green_avg_fare_per_season.csv", index=True)

In [ ]:
route_ddf = fhvhv_df[
    (fhvhv_df['pickup_lat'] == jfk_lat) & 
    (fhvhv_df['pickup_lon'] == jfk_lon) & 
    (fhvhv_df['dropoff_lat'] == ues_lat) & 
    (fhvhv_df['dropoff_lon'] == ues_lon)
]

In [ ]:
# Step 2: Extract year from pickup_datetime
route_ddf['year'] = route_ddf['pickup_datetime'].dt.year

In [ ]:
# Step 3: Group by year and compute average fare amount
avg_fare_per_year = route_ddf.groupby('year')['driver_pay'].mean().compute()

In [ ]:
# save to csv
avg_fare_per_year.to_csv(fhvhv_tables_path / "avg_fare_per_year.csv", index=True)

In [ ]:
# Define function to assign season
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

# Extract month and map to season
route_ddf['month'] = route_ddf['pickup_datetime'].dt.month
route_ddf['season'] = route_ddf['month'].map(get_season, meta=('month', 'str'))

# Group by season and compute average fare
avg_fare_per_season = route_ddf.groupby('season')['driver_pay'].mean().compute()

# Sort for better readability (optional)
avg_fare_per_season = avg_fare_per_season.loc[['Winter', 'Spring', 'Summer', 'Fall']]

# Display result
print(avg_fare_per_season)

In [ ]:
avg_fare_per_season.to_csv(fhvhv_tables_path / "avg_fare_per_season.csv", index=True)

In [ ]:
import matplotlib.pyplot as plt

# Example average fare data per season for each service
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
yellow_taxi = [54.24723387410412, 53.560678516209, 53.62137540695536, 54.137651310905326]
green_taxi = [42.333333333333336, 53.51428571428572,  60.65714285714286, 57.0]
fhvhv = [52.79222731513406, 56.467724381351324, 55.315100772440736, 54.521964639023366]

# Create the plot
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)

ax = axs[0]
# Plot each service
ax.plot(seasons, yellow_taxi, marker='o', label='Yellow', color="#ffee8c")
ax.plot(seasons, green_taxi, marker='o', label='Green', color="#77dd77")
ax.plot(seasons, fhvhv, marker='o', label='FHVHV', color="#aec7e8")

# Set labels
ax.set_xlabel('Season')
ax.set_ylabel('Average price')

# remove spines up and right
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# add grid for better readability
ax.grid(True, linestyle='--', alpha=0.3)
# Show legend
ax.legend()

yellow = pd.read_csv("/d/hpc/home/jv8043/BD/project/T4/T4_tables/yellow/yellow_avg_fare_per_year.csv")   # contains columns: year,fare_amount
green = pd.read_csv("/d/hpc/home/jv8043/BD/project/T4/T4_tables/green/green_avg_fare_per_year.csv")
fhvhv = pd.read_csv("/d/hpc/home/jv8043/BD/project/T4/T4_tables/fhvhv/avg_fare_per_year.csv")
# filter all three from 2019 to 2023
yellow = yellow[(yellow['year'] >= 2019)]
fhvhv = fhvhv[(fhvhv['year'] >= 2019)]
# Add service type to each DataFrame
yellow['service'] = 'Yellow'
yellow = yellow.sort_values(by='year')
fhvhv['service'] = 'FHVHV'
fhvhv = fhvhv.sort_values(by='year')
fhvhv = fhvhv.rename(columns={'driver_pay': 'fare_amount'})  # rename driver_pay to fare_amount for consistency

# Combine all into one DataFrame
df = pd.concat([yellow, fhvhv], ignore_index=True)

# Ensure year is int and sort by year
df['year'] = df['year'].astype(int)

# Pivot to get years as x and services as separate columns
pivot = df.pivot(index='year', columns='service', values='fare_amount')

ax = axs[1]
# Plot
ax.plot(pivot.index, pivot['Yellow'], marker='o', label='Yellow', color="#ffee8c")
# ax.plot(pivot.index, pivot['Green'], marker='o', label='Green', color="#77dd77")
ax.plot(pivot.index, pivot['FHVHV'], marker='o', label='FHVHV', color="#aec7e8")
print(pivot['FHVHV'])
ax.set_xticks(pivot.index)
ax.set_xticklabels(pivot.index, rotation=45)
ax.set_xlabel("Year")
# ax.set_ylabel("Average price")
ax.legend()
# remove spines up and right
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# add grid for better readability
ax.grid(True, linestyle='--', alpha=0.3)

# Display the plot
fig.tight_layout()
plt.show()
fig.savefig(vis_path / "jfk_ues_avg_fare_per_season.pdf", dpi=300, bbox_inches='tight')

In [ ]:
# Example average fare data per season for each service
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
yellow_taxi = [42.33, 53.51, 60.66, 57.00]
green_taxi = [42.333333333333336, 53.51428571428572,  60.65714285714286, 57.0]
fhvhv = [52.79222731513406, 56.467724381351324, 55.315100772440736, 54.521964639023366]

# Create the plot
fig, ax = plt.subplots(figsize=(4, 3))

# Plot each service
ax.plot(seasons, yellow_taxi, marker='o', label='Yellow Taxi')
ax.plot(seasons, green_taxi, marker='o', label='Green Taxi')
ax.plot(seasons, fhvhv, marker='o', label='FHVHV')

# Set labels
ax.set_xlabel('Season')
ax.set_ylabel('Average Fare Amount')

# Show legend
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()